# Task VI QML-HEP : Quantum representation learning
In this task you should implement a simple representation learning scheme based on a contrastive loss:
Load the MNIST dataset
Write a function which takes an image and prepares a quantum state. This function should have trainable parameters which we want to learn in order to have good quantum representations
Create a circuit with which takes two images and embeds both as quantum states with the function you wrote before. Afterwards the circuit should perform a SWAP test between the two states. In the end the measurement should give the fidelity of the quantum states.
Train the circuit parameters with a contrastive loss: For two MNIST images in the same class the fidelity should be maximized, while for images of different classes the fidelity should be minimized.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
1
Tesla T4


In [ ]:
# Load MNIST dataset
def get_mnist_loaders(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader


A function which takes an image and prepares a quantum state.

In [ ]:
# Quantum Embedding Layer
class QuantumEmbedding(nn.Module):
    def __init__(self, embedding_dim=32):  # Increased embedding dimension
        super().__init__()
        self.fc = nn.Linear(784, embedding_dim)  # Trainable parameters
        self.activation = nn.Tanh()  # Non-linearity to mimic quantum state

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten images
        return self.activation(self.fc(x))



Create a circuit which embeds two images as quantum states and performs a SWAP test.

In [ ]:
# SWAP test using Cosine Similarity
def swap_test(state1, state2):
    state1 = nn.functional.normalize(state1, p=2, dim=1)
    state2 = nn.functional.normalize(state2, p=2, dim=1)
    return torch.matmul(state1, state2.T)  # Compute full pairwise cosine similarity


In [ ]:
# Contrastive loss with margin
def contrastive_loss(fidelity, same_class, margin=0.5):
    fidelity = fidelity.view(-1)
    same_class = same_class.view(-1)
    pos_loss = (1 - fidelity) ** 2 * same_class
    neg_loss = torch.relu(fidelity - margin) ** 2 * (1 - same_class)  # Fixed inversion
    return torch.mean(pos_loss + neg_loss)


Train the circuit parameters with contrastive loss.

In [ ]:
# Train model
def train_model(train_loader, epochs=10, lr=0.001):
    embedding_dim = 32
    model = QuantumEmbedding(embedding_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0
        for (img1, label1), (img2, label2) in zip(train_loader, train_loader):
            batch_size = min(img1.shape[0], img2.shape[0])
            img1, img2 = img1[:batch_size].to(device), img2[:batch_size].to(device)
            label1, label2 = label1[:batch_size].to(device), label2[:batch_size].to(device)

            state1 = model(img1)
            state2 = model(img2)

            fidelity = swap_test(state1, state2)
            same_class = (label1.unsqueeze(1) == label2.unsqueeze(0)).float()  # Ensure correct shape
            loss = contrastive_loss(fidelity, same_class)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
    return model


In [ ]:
# Model evaluation
def evaluate_model(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for img1, label1 in test_loader:
            img1, label1 = img1.to(device), label1.to(device)
            state1 = model(img1)

            for img2, label2 in test_loader:
                batch_size = min(img1.shape[0], img2.shape[0])
                img2, label2 = img2[:batch_size].to(device), label2[:batch_size].to(device)
                state2 = model(img2)

                fidelity = swap_test(state1, state2)
                predicted = (fidelity > 0.5).float()
                same_class = (label1.unsqueeze(1) == label2.unsqueeze(0)).float()  # Ensure correct shape

                correct += (predicted == same_class).sum().item()
                total += same_class.numel()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")


In [ ]:
# Run training
train_loader, test_loader = get_mnist_loaders()
trained_model = train_model(train_loader)
evaluate_model(test_loader, trained_model)


Epoch 1, Loss: 0.011553901243708663
Epoch 2, Loss: 0.00855467302391111
Epoch 3, Loss: 0.007918420885163329
Epoch 4, Loss: 0.007501513936491346
Epoch 5, Loss: 0.0072756362542795985
Epoch 6, Loss: 0.0070444197167576885
Epoch 7, Loss: 0.006870254509842027
Epoch 8, Loss: 0.006752091714107533
Epoch 9, Loss: 0.006681959520080196
Epoch 10, Loss: 0.006554098465223747
Accuracy: 85.29%


## Why this approach?
* Instead of using fixed quantum circuits, we implement a QuantumEmbedding. This allows the embedding to learn an optimal feature representation for the MNIST dataset.
* Efficient Pairwise Fidelity Computation. Using swap_test() with cosine similarity provides an efficient approximation of quantum fidelity between embeddings. This avoids the complexity of directly simulating quantum circuits while maintaining meaningful comparisons.
* Contrastive Loss with Margin.
Maximizing fidelity for images of the same class.
Minimizing fidelity beyond a margin for images of different classes.
* Improved Training Stability. Gradient Clipping and Learning Rate Scheduler .
<br>
We are using free tier of notebooks which means achiving good accuracy, in limited time and limited resources is tough but using a good apparoch will help us build a good model with limited resources.